# Cell community identification using the hoodscanR package 

<img style="float: middle;" src="../images/logo.png">

**This script has been generated by Feng Zhang and Onkar Mulay from the *Genomics and Machine Learning Lab*.** 

If you have any questions, please do not hesitate to contact us (feng.zhang@uq.edu.au or o.mulay@uq.edu.au).

- [1 Introduction](#1-Introduction)<br>
- [2 Cell community identification](#2-Cell-community-identification)<br>
    - [2.1 Input data](#2.1-Input-data)<br>
    - [2.2 Perform neighborhood analysis](#Perform-neighborhood-analysis)<br>


## 1 Introduction 
### Why do we identify the cell community
To understand the structure-function relationship of a tissue, the concept of tissue cellular neighborhoods (TCNs), niches or cell community has been proposed as a recurrent functional unit in which different cell types organize themselves to support tissue functions. 

Understanding the complexities of cell community is critical for advancing our knowledge of tissue biology and developing novel therapeutic strategies for various diseases.

### What is CosMx data
[CosMx™ Spatial Molecular Imager](https://nanostring.com/products/cosmx-spatial-molecular-imager/single-cell-imaging-overview/) is a fully-integrated multiomics single-cell spatial biology solution and enables researchers to investigate morphologically-intact tissues at unprecedented plex, resolution and cell numbers.

### What tools can we use to run the cell community
- [hoodscanR](https://github.com/DavisLaboratory/hoodscanR)
- [NeighborhoodCoordination](https://github.com/nolanlab/NeighborhoodCoordination)
- [CytoCommunity](https://github.com/huBioinfo/CytoCommunity)
- [nichecompass](https://github.com/Lotfollahi-lab/nichecompass)
- [SPIAT](https://github.com/TrigosTeam/SPIAT)
- 

### What is hoodscanR
hoodscanR is an user-friendly R package providing functions to assist cellular neighborhood analysis of any spatial transcriptomics data with single-cell resolution.

All functions in the package are built based on the *SpatialExperiment* infrastructure, allowing integration into various spatial transcriptomics-related packages from Bioconductor. The package can result in cell-level neighborhood annotation output, along with funtions to perform neighborhood colocalization analysis and neighborhood-based cell clustering.

![image.png](images/hoodscanR.jpg)


## 2 Cell community identification
### 2.1 Input data 

In [ ]:
# library and set default parameter 
library(hoodscanR)
library(SpatialExperiment)
library(png)

In [ ]:
## This code will help speed up the plotting when running in base R on a Mac
show_plot <- function(plot, filename = "temp.png", width = 1600, height = 1500, res = 260) {                                                  
  temp_filename <- tempfile(fileext = ".png")
  grDevices::png(filename = temp_filename, width = width, height = height, res = res)
  print(plot)
  dev.off()
  img <- png::readPNG(temp_filename)
  grid::grid.raster(img)
}

In [ ]:
# set path and load data
#data_dir <- '/scratch/project/stseq/Feng/package-vignette/temp-data'
data_dir <- '/data/module2/data/'
set.seed(123)

In [ ]:
# load and explore the CosMx data
load(file.path(data_dir,'CosMx_Skin_Melanoma.RData'))
print(dim(count))
print(dim(spatial))


In [ ]:
count[1:5,1:5]

In [ ]:
head(spatial)

In [ ]:
# plot the cell position on the tissue which cover 12 field of view (FOVs) 
spe <- SpatialExperiment(
    assay = list(counts=count), 
    colData = spatial, 
    rowData = rownames(count),
    spatialCoordsNames = c("sdimx", "sdimy"))
p1 <- plotTissue(spe, color = custom_clust, size = 0.5, alpha = 0.8)
show_plot(p1)

In [ ]:
# take fov 12 for example
count_fov <- count[,spatial$fov==12]
spatial_fov <- spatial[spatial$fov==12,]
dim(count_fov)
dim(spatial_fov)

In [ ]:
head(spatial_fov)

In [ ]:
spe <- SpatialExperiment(
    assay = list(counts=count_fov), 
    colData = spatial_fov, 
    rowData = rownames(count_fov),
    spatialCoordsNames = c("sdimx", "sdimy"))
p2 <- plotTissue(spe, color = custom_clust, size = 1.5, alpha = 0.8)
show_plot(p2)

### Perform neighborhood analysis

In [ ]:
spe <- readHoodData(spe, anno_col = "custom_clust")
fnc <- findNearCells(spe, k = 100) # find 100 nearest cells for each cell
lapply(fnc, function(x) x[1:10, 1:5])

In [ ]:
# perform neighborhood analysis to genereate a matrix 
# with the probability of each cell associating with their 100 nearest cells.
pm <- scanHoods(fnc$distance) #

hoods <- mergeByGroup(pm, fnc$cells)
p3 <- plotHoodMat(hoods, n = 10, hm_height = 5,hm_width = 8)
show_plot(p3)

In [ ]:
spe <- mergeHoodSpe(spe, hoods)
spe <- calcMetrics(spe, pm_cols = colnames(hoods))
spe <- clustByHood(spe, pm_cols = colnames(hoods), k = 10)
p4 <- plotTissue(spe, color = clusters)
show_plot(p4)

In [ ]:
# entropy and perplexity measure the mixture of neighborhood of each cell
p5 <- plotTissue(spe, size = 1.5, color = perplexity) 
show_plot(p5)

In [ ]:
p6 <- plotProbDist(spe, pm_cols = colnames(hoods), by_cluster = TRUE, plot_all = TRUE, show_clusters = as.character(seq(10)))
show_plot(p6)

In [ ]:
# write.csv(spatial,file = 'spatial.csv',row.names=F,col.names=T,quote=F)